In [29]:
from keras.models import Model
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.pyplot import imshow
from PIL import Image

from keras.datasets import cifar10

imgPath = "/mnt/ml/NOAA/color"
irPath = "/mnt/ml/NOAA/thermal"
dataPath = "/mnt/ml/NOAA/arcticseals/data"

### Metaparameters

In [2]:
NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
VALIDATION_SPLIT = 0.2

### Data generator

In [3]:
def numpy_crop(img, x1, y1, x2, y2):
    return img[y1:y2, x1:x2]

In [59]:
# TODO: https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly.html
def SealDataGenerator():
    df = pd.read_csv("{}/training.csv".format(dataPath))
    rowcount = int(df.shape[0])
    train_count = int(rowcount * (1 - VALIDATION_SPLIT))
    test_count = rowcount - train_count
    x_train = np.zeros([train_count, 512, 512, 3])
    x_test = np.zeros([test_count, 512, 512, 3])
    y_train = np.zeros([train_count, 1])
    y_test = np.zeros([test_count, 1])
    
    
    i = 0
    for index, row in df[:train_count, :]:
        imageFile = "{}/{}".format(imgPath, row.filt_color)
        image = Image.open(imageFile)
        
        # TODO: Some images have negative bounding boxes.
        # print((imageFile, row.thumb_left, row.thumb_top, row.thumb_right, row.thumb_bottom))
        if (row.thumb_left < 0 or row.thumb_top < 0 or row.thumb_right < 0 or row.thumb_bottom < 0):
            print("x", end="")
        else:
            subimage = image.crop((row.thumb_left, row.thumb_top, row.thumb_right, row.thumb_bottom))
            npsubimage = np.array(subimage)
            
            print (npsubimage.shape)
            x_train[i] = npsubimage
            print (train)
            print(".", end="")
            
        break;
        
        i+=1
        
    print("Images processed {}".format(i))

In [60]:
SealDataGenerator()

TypeError: unhashable type: 'slice'

In [35]:
(x_t, y_t), (x_test, y_test) = cifar10.load_data()
print (x_t.shape)
print (y_t.shape)

(50000, 32, 32, 3)
(50000, 1)


VGG16 model layout:

In [5]:
def ShowLayers(base_model):
    for i, layer in enumerate(base_model.layers):
        print (i, layer.name, layer.output_shape)

In [6]:
original_model = VGG16(weights=None, include_top=True)
ShowLayers(original_model)

0 input_1 (None, 224, 224, 3)
1 block1_conv1 (None, 224, 224, 64)
2 block1_conv2 (None, 224, 224, 64)
3 block1_pool (None, 112, 112, 64)
4 block2_conv1 (None, 112, 112, 128)
5 block2_conv2 (None, 112, 112, 128)
6 block2_pool (None, 56, 56, 128)
7 block3_conv1 (None, 56, 56, 256)
8 block3_conv2 (None, 56, 56, 256)
9 block3_conv3 (None, 56, 56, 256)
10 block3_pool (None, 28, 28, 256)
11 block4_conv1 (None, 28, 28, 512)
12 block4_conv2 (None, 28, 28, 512)
13 block4_conv3 (None, 28, 28, 512)
14 block4_pool (None, 14, 14, 512)
15 block5_conv1 (None, 14, 14, 512)
16 block5_conv2 (None, 14, 14, 512)
17 block5_conv3 (None, 14, 14, 512)
18 block5_pool (None, 7, 7, 512)
19 flatten (None, 25088)
20 fc1 (None, 4096)
21 fc2 (None, 4096)
22 predictions (None, 1000)


Removing top and last 3 layers:

In [7]:
# Input = 512x512x3
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(512,512,3)) # Will download the 560MB trained model.

# Output = ['Anomaly' 'Bearded Seal' 'Ringed Seal' 'Polar Bear' 'UNK Seal']
x = base_model.output
x = Flatten(name='flatten')(x)
x = Dense(4096, name='fc1')(x)
x = Dense(4096, name='fc2')(x)
predictions = Dense(5, activation='softmax', name='predictions')(x)

model = Model(inputs=base_model.input, outputs=predictions)
ShowLayers(model)

0 input_2 (None, 512, 512, 3)
1 block1_conv1 (None, 512, 512, 64)
2 block1_conv2 (None, 512, 512, 64)
3 block1_pool (None, 256, 256, 64)
4 block2_conv1 (None, 256, 256, 128)
5 block2_conv2 (None, 256, 256, 128)
6 block2_pool (None, 128, 128, 128)
7 block3_conv1 (None, 128, 128, 256)
8 block3_conv2 (None, 128, 128, 256)
9 block3_conv3 (None, 128, 128, 256)
10 block3_pool (None, 64, 64, 256)
11 block4_conv1 (None, 64, 64, 512)
12 block4_conv2 (None, 64, 64, 512)
13 block4_conv3 (None, 64, 64, 512)
14 block4_pool (None, 32, 32, 512)
15 block5_conv1 (None, 32, 32, 512)
16 block5_conv2 (None, 32, 32, 512)
17 block5_conv3 (None, 32, 32, 512)
18 block5_pool (None, 16, 16, 512)
19 flatten (None, 131072)
20 fc1 (None, 4096)
21 fc2 (None, 4096)
22 predictions (None, 5)


### Freeze VGG layers

In [9]:
for layer in base_model.layers: layer.trainable = False

In [10]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')